In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import string
import random

In [2]:
KEYWORD="CRYPTO"
ALPHABET = string.ascii_uppercase.replace('J', '')  # 'J' is usually omitted
ALPHABET_SIZE = len(ALPHABET)

In [3]:
def generate_playfair_matrix(keyword):
    seen = set()
    matrix = []
    for char in keyword.upper():
        if char not in seen and char in ALPHABET:
            seen.add(char)
            matrix.append(char)
    for char in ALPHABET:
        if char not in seen:
            matrix.append(char)
    return matrix

In [4]:
def prepare_pairs(plaintext):
    plaintext = plaintext.upper().replace(" ", "").replace("J", "I")
    pairs = []
    i = 0
    while i < len(plaintext):
        a = plaintext[i]
        if i + 1 < len(plaintext):
            b = plaintext[i + 1]
            if a == b:
                pairs.append((a, 'X'))  # Same letters, add 'X'
                i += 1
            else:
                pairs.append((a, b))
                i += 2
        else:
            pairs.append((a, 'X'))  # If odd, add 'X'
            i += 1
    return pairs

In [5]:
def encrypt_playfair(plaintext, keyword):
    matrix = generate_playfair_matrix(keyword)
    pairs = prepare_pairs(plaintext)
    ciphertext = ''

    # Create a mapping for positions in the matrix
    position = {char: (i // 5, i % 5) for i, char in enumerate(matrix)}

    for a, b in pairs:
        row_a, col_a = position[a]
        row_b, col_b = position[b]
        
        if row_a == row_b:  # Same row
            ciphertext += matrix[row_a * 5 + (col_a + 1) % 5]
            ciphertext += matrix[row_b * 5 + (col_b + 1) % 5]
        elif col_a == col_b:  # Same column
            ciphertext += matrix[((row_a + 1) % 5) * 5 + col_a]
            ciphertext += matrix[((row_b + 1) % 5) * 5 + col_b]
        else:  # Rectangle
            ciphertext += matrix[row_a * 5 + col_b]
            ciphertext += matrix[row_b * 5 + col_a]
    
    return ciphertext

In [6]:
def generate_data(num_samples=1000, max_length=10):
    data_list = []
    for _ in range(num_samples):
        length = random.randint(1, max_length)
        plaintext = ''.join(random.choices(ALPHABET, k=length))
        keyword = KEYWORD  # Random keyword length
        ciphertext = encrypt_playfair(plaintext, keyword)

        # Create node features (one-hot encoding)
        x = []
        for char in plaintext:
            one_hot = [0] * ALPHABET_SIZE
            one_hot[ALPHABET.index(char)] = 1
            x.append(one_hot)
        x = torch.tensor(x, dtype=torch.float)

        # Edge index: connect each character to the next
        edge_index = []
        for i in range(len(plaintext) - 1):
            edge_index.append([i, i + 1])
            edge_index.append([i + 1, i])  # Undirected edges
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

        # Prepare labels
        y = []
        pairs = prepare_pairs(plaintext)
        for a, b in pairs:
            encrypted_pair = encrypt_playfair(a + b, keyword)
            y.extend([ALPHABET.index(char) for char in encrypted_pair])

        y = torch.tensor(y, dtype=torch.long)

        # Ensure the output matches the number of nodes (adjust if needed)
        if len(y) != len(x):  # If lengths don't match
            continue  # Skip this sample to avoid mismatches

        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)
    return data_list


In [7]:
dataset = generate_data(num_samples=2000, max_length=10)
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\ekru\AppData\Roaming\Python\Python312\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [8]:
class CipherGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CipherGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x

In [9]:
# Hyperparameters
INPUT_DIM = ALPHABET_SIZE
HIDDEN_DIM = 64
OUTPUT_DIM = ALPHABET_SIZE
EPOCHS = 50
LEARNING_RATE = 0.01

In [10]:
model = CipherGNN(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [11]:
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        # Reshape outputs and labels for loss computation
        out = out.view(-1, OUTPUT_DIM)
        y = batch.y.view(-1)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")

Epoch 1, Loss: 3.0353
Epoch 2, Loss: 2.2446
Epoch 3, Loss: 1.8511
Epoch 4, Loss: 1.6752
Epoch 5, Loss: 1.5690
Epoch 6, Loss: 1.5152
Epoch 7, Loss: 1.4668
Epoch 8, Loss: 1.4159
Epoch 9, Loss: 1.3906
Epoch 10, Loss: 1.3511
Epoch 11, Loss: 1.3221
Epoch 12, Loss: 1.2887
Epoch 13, Loss: 1.2690
Epoch 14, Loss: 1.2358
Epoch 15, Loss: 1.2013
Epoch 16, Loss: 1.1657
Epoch 17, Loss: 1.1510
Epoch 18, Loss: 1.1512
Epoch 19, Loss: 1.1295
Epoch 20, Loss: 1.0953
Epoch 21, Loss: 1.0775
Epoch 22, Loss: 1.0520
Epoch 23, Loss: 1.0259
Epoch 24, Loss: 1.0067
Epoch 25, Loss: 0.9943
Epoch 26, Loss: 0.9699
Epoch 27, Loss: 0.9500
Epoch 28, Loss: 0.9363
Epoch 29, Loss: 0.9233
Epoch 30, Loss: 0.9211
Epoch 31, Loss: 0.8943
Epoch 32, Loss: 0.8916
Epoch 33, Loss: 0.8610
Epoch 34, Loss: 0.8588
Epoch 35, Loss: 0.8406
Epoch 36, Loss: 0.8295
Epoch 37, Loss: 0.8282
Epoch 38, Loss: 0.8166
Epoch 39, Loss: 0.7858
Epoch 40, Loss: 0.7710
Epoch 41, Loss: 0.7490
Epoch 42, Loss: 0.7514
Epoch 43, Loss: 0.7470
Epoch 44, Loss: 0.72

In [12]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        out = model(batch)
        preds = out.argmax(dim=1)
        correct += (preds == batch.y).sum().item()
        total += batch.y.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 51.23%


In [13]:
def encrypt_with_model(model, plaintext, keyword=KEYWORD):
    model.eval()
    # Prepare node features
    x = []
    for char in plaintext.upper():
        one_hot = [0] * ALPHABET_SIZE
        one_hot[ALPHABET.index(char)] = 1
        x.append(one_hot)
    x = torch.tensor(x, dtype=torch.float).unsqueeze(0)  # Batch size 1

    # Create edge index
    edge_index = []
    for i in range(len(plaintext) - 1):
        edge_index.append([i, i + 1])
        edge_index.append([i + 1, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().unsqueeze(0)

    # Create batch
    data = Data(x=x.squeeze(0), edge_index=edge_index.squeeze(0))
    out = model(data)
    preds = out.argmax(dim=1)
    ciphertext = ''.join([ALPHABET[p.item()] for p in preds])
    return ciphertext

In [14]:
sample_plaintext = "KKMFURIK"
predicted_ciphertext = encrypt_with_model(model, sample_plaintext, KEYWORD)
print(f"Plaintext: {sample_plaintext}")
print(f"Predicted Ciphertext: {predicted_ciphertext}")
print(f"Actual Ciphertext: {encrypt_playfair(sample_plaintext, KEYWORD)}")

Plaintext: KKMFURIK
Predicted Ciphertext: FFGLCGFF
Actual Ciphertext: IZGSLCPGIZ
